In [ ]:
import os
import subprocess

# Set up FSL environment
fsl_path = "/home/a1/fsl"
os.environ["FSLDIR"] = fsl_path
os.environ["PATH"] += os.pathsep + os.path.join(fsl_path, "bin")
os.environ["FSLOUTPUTTYPE"] = "NIFTI_GZ"

# Define base directory where the BIDS data is stored
BASE_DIR = "/home/a1/Desktop/Projects/forkx/HC_nii"
FREE_DIR = "/home/a1/Desktop/Projects/forkx/HC_nii"
# Path to the list of subjects file
SUBJECT_LIST = "/home/a1/Desktop/Projects/code/hc_list.txt"

# Read the list of subjects
with open(SUBJECT_LIST, 'r') as file:
    subjects = file.read().splitlines()

# Loop over each subject in the list
for SUBJ in subjects:
    SUBJ_ID = f"sub-{SUBJ}"
    print(f"Processing {SUBJ_ID}")

    # Define subject directory
    SUBJ_DIR = os.path.join(BASE_DIR, SUBJ_ID)

    # Define file paths
    t1_fname = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_brainmask_in_space.nii.gz")
    label_fname = os.path.join(FREE_DIR, f"{SUBJ}","mri", "wm_bin.nii.gz")
    MAT = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_brainmask_to_dwi.mat")
    BVAL = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi.bval")
    BVEC = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi.bvec")
    WM = os.path.join(FREE_DIR, f"{SUBJ}","mri", "wm_in_dwi.nii.gz")
    EDDY_IMAGE = os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi_brain.nii.gz")
    OUT=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_b0_brain_in_MNI.nii.gz")
    WM = os.path.join(FREE_DIR, f"{SUBJ}","mri", "wm_in_dwi.nii.gz")
    B0=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_b0.nii.gz")
    TEM='/home/a1/Desktop/Projects/template/MNI152_T1_2mm_brain.nii.gz'
    eddy_OUTPUT=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_dwi_in_MNI.nii.gz")
    WARP=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_b0_brain_in_MNI.nii.gz1Warp.nii.gz") 
    TransMAT=os.path.join(SUBJ_DIR, 'dwi', f"{SUBJ_ID}_b0_brain_in_MNI.nii.gz0GenericAffine.mat") 
    # Run BET to extract brain mask
    convert_cmd = ['fslroi',EDDY_IMAGE, B0,  '0', '1']
    try:
        subprocess.run(convert_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BET: {e}")
        continue
    
   
    # Run BET to extract brain mask
    ant_cmd = ['antsRegistrationSyNQuick.sh','-d','3','-f',TEM,'-m',B0,'-o', OUT]
    try:
        subprocess.run(ant_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BET: {e}")
        continue

    
     # Run BET to extract brain mask
    anttr_cmd = ['antsApplyTransforms','-d','3','-i',EDDY_IMAGE,'-r',TEM,'-o', eddy_OUTPUT,'-t', WARP, '-t', TransMAT, '--float']
    try:
        subprocess.run(anttr_cmd, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error running BET: {e}")
        continue
    


  #  data, affine, hardi_img = load_nifti(EDDY_IMAGE, return_img=True)
   # labels = load_nifti_data(label_fname)
   # t1_data = load_nifti_data(t1_fname)
   # bvals, bvecs = read_bvals_bvecs(BVAL, BVEC)
   # gtab = gradient_table(bvals, bvecs)

 